In [27]:
import numpy as np
import pandas as pd
import os

import albumentations
import cv2
import timm
import torch.nn as nn
from sklearn import metrics
import torch
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

import pytorch_lightning as pl
from pytorch_lightning import callbacks
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning import LightningDataModule

import glob
import matplotlib.pyplot as plt

from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold, train_test_split

import clip
from PIL import Image
from joblib import dump, load
from timm.data.transforms_factory import create_transform
from timm.data import resolve_data_config

import time
import gc

In [2]:
SEED = 2023

DATA_DIR = 'data'

TRAIN_IMAGES_DIR = os.path.join(DATA_DIR, 'train')
TEST_IMAGES_DIR = os.path.join(DATA_DIR, 'test')

OUTPUT_DIR = "output"

In [3]:
df_train = pd.read_csv(os.path.join(DATA_DIR, 'train.csv'))
df_test = pd.read_csv(os.path.join(DATA_DIR, 'test.csv'))

df_train['path'] = df_train['Id'].map(lambda x: os.path.join(TRAIN_IMAGES_DIR, f'{x}.jpg'))
df_test['path'] = df_test['Id'].map(lambda x: os.path.join(TEST_IMAGES_DIR, f'{x}.jpg'))

In [4]:
def create_folds(data: pd.DataFrame, num_splits = 5):
    data["fold"] = -1
    num_bins = int(np.floor(1 + np.log2(len(data)))) # sturge's rule

    data.loc[:, "bins"] = pd.cut(data["Pawpularity"], bins=num_bins, labels=False)
    skf = StratifiedKFold(n_splits=num_splits, shuffle=True, random_state=SEED)

    for fold_index, (train_idx, val_idx) in enumerate(skf.split(X=data, y=data.bins.values)):
        data.loc[val_idx, 'fold'] = fold_index

    data = data.drop('bins', axis=1)

    return data

In [5]:
df_train_5 = create_folds(df_train)

# Extract Image Embeddings from Each Architecture

In [6]:
names = [
    'deit_base_distilled_patch16_384',
    'ig_resnext101_32x48d',
    'repvgg_b0',
    'resnetv2_152x4_bitm',
    'swsl_resnext101_32x8d',
    'tf_efficientnet_l2_ns_475',
    'vit_base_patch16_384',
    'vit_large_r50_s32_384',
]

In [7]:
class PawpularDataset:
    def __init__(self, image_paths, modelcfg=None):
        self.image_paths = image_paths
        self.transform = create_transform(**modelcfg)
        
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, item):
        img = Image.open(self.image_paths[item]).convert('RGB')
        img = self.transform(img)
        return img

In [12]:
def create_embeddings(df: pd.DataFrame, model_names: list[str]):
    embeddings = {}
    for model_name in model_names:
        starttime = time.time()
        model = timm.create_model(model_name=model_name, pretrained=True).to('cuda')
        model.eval()

        train_dataset = PawpularDataset(image_paths=df['path'].values, modelcfg=resolve_data_config({}, model=model))
        train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=False)

        print(model_name, resolve_data_config({}, model=model_name))

        with torch.no_grad():
            result = [model(img.to('cuda')).cpu().numpy() for img in train_dataloader]
        result = np.concatenate(result, 0)

        embeddings[model_name] = result
        print(f"{model_name}: {int(time.time() - starttime)}s")

    return embeddings

In [13]:
train_embeddings = create_embeddings(df_train, names)

deit_base_distilled_patch16_384 {'input_size': (3, 224, 224), 'interpolation': 'bicubic', 'mean': (0.485, 0.456, 0.406), 'std': (0.229, 0.224, 0.225), 'crop_pct': 0.875, 'crop_mode': 'center'}
deit_base_distilled_patch16_384: 1193s


d:\Desktop\Programming\CZ4041-Project\venv\lib\site-packages\timm\models\_factory.py:114: UserWarning: Mapping deprecated model name ig_resnext101_32x48d to current resnext101_32x8d.fb_wsl_ig1b_ft_in1k.
  model = create_fn(


d:\Desktop\Programming\CZ4041-Project\venv\lib\site-packages\huggingface_hub\file_download.py:137: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\asdf\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


ig_resnext101_32x48d {'input_size': (3, 224, 224), 'interpolation': 'bicubic', 'mean': (0.485, 0.456, 0.406), 'std': (0.229, 0.224, 0.225), 'crop_pct': 0.875, 'crop_mode': 'center'}
ig_resnext101_32x48d: 406s


repvgg_b0 {'input_size': (3, 224, 224), 'interpolation': 'bicubic', 'mean': (0.485, 0.456, 0.406), 'std': (0.229, 0.224, 0.225), 'crop_pct': 0.875, 'crop_mode': 'center'}
repvgg_b0: 133s


...

swsl_resnext101_32x8d {'input_size': (3, 224, 224), 'interpolation': 'bicubic', 'mean': (0.485, 0.456, 0.406), 'std': (0.229, 0.224, 0.225), 'crop_pct': 0.875, 'crop_mode': 'center'}
swsl_resnext101_32x8d: 314s


d:\Desktop\Programming\CZ4041-Project\venv\lib\site-packages\timm\models\_factory.py:114: UserWarning: Mapping deprecated model name tf_efficientnet_l2_ns_475 to current tf_efficientnet_l2.ns_jft_in1k_475.
  model = create_fn(


tf_efficientnet_l2_ns_475 {'input_size': (3, 224, 224), 'interpolation': 'bicubic', 'mean': (0.485, 0.456, 0.406), 'std': (0.229, 0.224, 0.225), 'crop_pct': 0.875, 'crop_mode': 'center'}
tf_efficientnet_l2_ns_475: 2398s


vit_base_patch16_384 {'input_size': (3, 224, 224), 'interpolation': 'bicubic', 'mean': (0.485, 0.456, 0.406), 'std': (0.229, 0.224, 0.225), 'crop_pct': 0.875, 'crop_mode': 'center'}
vit_base_patch16_384: 705s


vit_large_r50_s32_384 {'input_size': (3, 224, 224), 'interpolation': 'bicubic', 'mean': (0.485, 0.456, 0.406), 'std': (0.229, 0.224, 0.225), 'crop_pct': 0.875, 'crop_mode': 'center'}
vit_large_r50_s32_384: 790s


In [17]:
dump(train_embeddings, os.path.join(OUTPUT_DIR, 'train_embeddings.joblib'))

['output\\train_embeddings.joblib']

In [21]:
train_embeddings = load(os.path.join(OUTPUT_DIR, 'train_embeddings.joblib'))

In [22]:
for m in train_embeddings.keys():
    print(f"{m}: {train_embeddings[m].shape}")

deit_base_distilled_patch16_384: (9912, 1000)
ig_resnext101_32x48d: (9912, 1000)
repvgg_b0: (9912, 1000)
resnetv2_152x4_bitm: (9912, 1000)
swsl_resnext101_32x8d: (9912, 1000)
tf_efficientnet_l2_ns_475: (9912, 1000)
vit_base_patch16_384: (9912, 1000)
vit_large_r50_s32_384: (9912, 1000)


In [54]:
for model_name in train_embeddings.keys():
    model = timm.create_model(model_name=model_name, pretrained=True)
    torch.save(model.state_dict(), os.path.join(OUTPUT_DIR, 'base_models', f'{model_name}.ckpt'))

d:\Desktop\Programming\CZ4041-Project\venv\lib\site-packages\timm\models\_factory.py:114: UserWarning: Mapping deprecated model name ig_resnext101_32x48d to current resnext101_32x8d.fb_wsl_ig1b_ft_in1k.
  model = create_fn(
d:\Desktop\Programming\CZ4041-Project\venv\lib\site-packages\timm\models\_factory.py:114: UserWarning: Mapping deprecated model name resnetv2_152x4_bitm to current resnetv2_152x4_bit.goog_in21k_ft_in1k.
  model = create_fn(
d:\Desktop\Programming\CZ4041-Project\venv\lib\site-packages\timm\models\_factory.py:114: UserWarning: Mapping deprecated model name swsl_resnext101_32x8d to current resnext101_32x8d.fb_swsl_ig1b_ft_in1k.
  model = create_fn(
d:\Desktop\Programming\CZ4041-Project\venv\lib\site-packages\timm\models\_factory.py:114: UserWarning: Mapping deprecated model name tf_efficientnet_l2_ns_475 to current tf_efficientnet_l2.ns_jft_in1k_475.
  model = create_fn(


In [50]:
def fit_svr(df_train: pd.DataFrame, embeddings_train: np.array, df_test: pd.DataFrame, embeddings_test: np.array):
    train_predictions = np.zeros(df_train.shape[0])
    test_predictions = np.zeros(df_test.shape[0])

    n_folds = df_train['fold'].max() + 1
    for fold in tqdm(range(n_folds)):
        train_idx = df_train['fold'] != fold
        val_idx = df_train['fold'] == fold

        model = make_pipeline(StandardScaler(), SVR(C=16.0, kernel='rbf', degree=3, max_iter=10000))
        model.fit(train_embeddings[train_idx], df_train['Pawpularity'][train_idx])

        train_predictions[val_idx] = np.clip(model.predict(train_embeddings[val_idx]), 1, 100)
        test_predictions += np.clip(model.predict(embeddings_test), 1, 100)

        del model
        gc.collect()

    test_predictions /= n_folds

    return train_predictions, test_predictions

In [51]:
for col in names:
    predictions = fit_svr(df_train, train_embeddings[col].copy())
    print(f"{col}: {mean_squared_error(df_train['Pawpularity'], predictions, squared=False)}")

  0%|          | 0/5 [00:00<?, ?it/s]d:\Desktop\Programming\CZ4041-Project\venv\lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
 20%|██        | 1/5 [00:28<01:55, 28.75s/it]d:\Desktop\Programming\CZ4041-Project\venv\lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
 40%|████      | 2/5 [00:57<01:25, 28.58s/it]d:\Desktop\Programming\CZ4041-Project\venv\lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
 60%|██████    | 3/5 [01:25<00:57, 28.66s/it]d:\Desktop\Programming\CZ4041-Project\venv\lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver 

deit_base_distilled_patch16_384: 17.746481730444195


  0%|          | 0/5 [00:00<?, ?it/s]d:\Desktop\Programming\CZ4041-Project\venv\lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
 20%|██        | 1/5 [00:28<01:54, 28.65s/it]d:\Desktop\Programming\CZ4041-Project\venv\lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
 40%|████      | 2/5 [00:57<01:26, 28.72s/it]d:\Desktop\Programming\CZ4041-Project\venv\lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
 60%|██████    | 3/5 [01:26<00:57, 28.68s/it]d:\Desktop\Programming\CZ4041-Project\venv\lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver 

ig_resnext101_32x48d: 17.838338085632664


  0%|          | 0/5 [00:00<?, ?it/s]d:\Desktop\Programming\CZ4041-Project\venv\lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
 20%|██        | 1/5 [00:28<01:55, 28.89s/it]d:\Desktop\Programming\CZ4041-Project\venv\lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
 40%|████      | 2/5 [00:57<01:26, 28.90s/it]d:\Desktop\Programming\CZ4041-Project\venv\lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
 60%|██████    | 3/5 [01:26<00:57, 28.98s/it]d:\Desktop\Programming\CZ4041-Project\venv\lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver 

...

tf_efficientnet_l2_ns_475: 17.621301474475345


  0%|          | 0/5 [00:00<?, ?it/s]d:\Desktop\Programming\CZ4041-Project\venv\lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
 20%|██        | 1/5 [00:28<01:52, 28.06s/it]d:\Desktop\Programming\CZ4041-Project\venv\lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
 40%|████      | 2/5 [00:55<01:22, 27.47s/it]d:\Desktop\Programming\CZ4041-Project\venv\lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
 60%|██████    | 3/5 [01:22<00:54, 27.35s/it]d:\Desktop\Programming\CZ4041-Project\venv\lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver 

vit_base_patch16_384: 17.912905349526373


  0%|          | 0/5 [00:00<?, ?it/s]d:\Desktop\Programming\CZ4041-Project\venv\lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
 20%|██        | 1/5 [00:28<01:52, 28.15s/it]d:\Desktop\Programming\CZ4041-Project\venv\lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
 40%|████      | 2/5 [00:56<01:25, 28.38s/it]d:\Desktop\Programming\CZ4041-Project\venv\lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
 60%|██████    | 3/5 [01:24<00:56, 28.10s/it]d:\Desktop\Programming\CZ4041-Project\venv\lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver 

vit_large_r50_s32_384: 18.010305929024177


In [52]:
concatenated_embeddings = np.concatenate([train_embeddings[k] for k in names], 1)
predictions = fit_svr(df_train, concatenated_embeddings.copy())

  0%|          | 0/5 [00:00<?, ?it/s]d:\Desktop\Programming\CZ4041-Project\venv\lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
 20%|██        | 1/5 [03:37<14:28, 217.22s/it]d:\Desktop\Programming\CZ4041-Project\venv\lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
 40%|████      | 2/5 [07:16<10:55, 218.44s/it]d:\Desktop\Programming\CZ4041-Project\venv\lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
 60%|██████    | 3/5 [10:59<07:20, 220.44s/it]d:\Desktop\Programming\CZ4041-Project\venv\lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solv

Overall MSE: 17.315638205844333


In [53]:
print(f"Overall MSE: {mean_squared_error(df_train['Pawpularity'], predictions, squared=False)}")

Overall MSE: 17.315638205844333


In [55]:
pd.__version__

'2.1.1'